# Run.c

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karpathy/llama2.c/blob/master/run.ipynb)

More details can be found in the [README.md](README.md) .

In [1]:
#@title Clone Project
!rm -rf tem-force
!git clone https://github.com/yyj6666667/tem-force
!ls
%cd tem-force

Cloning into 'tem-force'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 36), reused 91 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 1.08 MiB | 29.13 MiB/s, done.
Resolving deltas: 100% (36/36), done.
sample_data  tem-force
/content/tem-force


In [2]:
#@title Pick Your Model

#@markdown Choose model
model = "stories42M" #@param ["stories15M", "stories42M", "stories110M"]

download_url = ""

if(model == "stories15M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin"
if(model == "stories42M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin"
if(model == "stories110M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin"

print(f"download_url: {download_url}")

!wget $download_url

model_file = model + ".bin"

download_url: https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
--2026-01-10 14:27:38--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.97, 13.35.202.121, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/64c19eb0275dc8cd7f170b8f/20ab6952f8b77338484a45f788d50ed30dfda285f9954758b75d7deefd120d26?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27stories42M.bin%3B+filename%3D%22stories42M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1768058858&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MDU4ODU4fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjRjMTllYjAyNzVkYzhjZDdmMTcwYjhmLzIwYWI2OTUyZjhiNzczMzg0ODRhNDVmNzg4ZDUwZWQzMGRmZGEyODVmOTk1NDc1OGI3NWQ3ZGVlZm

In [42]:
#CPU;
!make runfast

gcc -Ofast -o run run.c -lm
gcc -Ofast -o runq runq.c -lm


若使用 CUDA 优化，需执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm命令编译代码；若使用了 cuBLAS 库，则执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm -L/usr/local/lib64 -lcublas编译代码；若采用 AVX、OpenMP 等其他优化方法，则直接执行make run即可。

In [3]:
#GPU
#!nvcc -O3 -o run run.cu -lcudart -lm

!nvcc -O3 -std=c++14 -arch=sm_75 -o run run_version1.cu

run_version1.cu(586): warning #2464-D: conversion from a string literal to "char *" is deprecated
          int dummy_prefix = str_lookup(" ", t->sorted_vocab, t->vocab_size);
                                        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

run_version1.cu(833): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *empty_prompt = "";
                           ^

run_version1.cu(1013): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *tokenizer_path = "tokenizer.bin";
                             ^

run_version1.cu(1019): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *mode = "generate";
                   ^

run_version1.cu(586): warning #2464-D: conversion from a string literal to "char *" is deprecated
          int dummy_prefix = str_lookup(" ", t->sorted_vocab, t->vocab_size);
                                      

In [4]:
#tool prepare
!git clone https://github.com/brendangregg/FlameGraph 2>/dev/null || echo 
"FlameGraph Already cloned"
!apt-get install -y cuda-nsight-systems-12-2 >/dev/null || echo "nsys is installed"

!nvprof --version
!nsys --version

nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2024 NVIDIA Corporation
Release version 12.5.82 (21)
NVIDIA Nsight Systems version 2023.2.3.1004-33186433v0


In [14]:
#备选方案nvprof

# 2. 运行 nvprof
!nvprof --print-gpu-summary ./run stories42M.bin  2> nvprof_output.txt
# 步骤 1: 提取 GPU activities 部分
!cat nvprof_output.txt

One day, a little boy named Tim found a scooter in his yard. He was so happy. Tim wanted to ride the scooter all day. He did not want to share it with his friends. Tim was being selfish.
Tim rode the scooter down the hill. He went very fast. But then, something unexpected happened. The scooter began to move on its own! Tim was so surprised. He stopped and tried to ride again. But the scooter was still moving.
Tim called his friends to help. They all pushed the scooter together. The scooter started to go faster. Tim was happy. He learned that sharing can be fun. From that day on, Tim was not selfish anymore.
==6544== NVPROF is profiling process 6544, command: ./run stories42M.bin
achieved tok/s: 21.552915
==6544== Profiling application: ./run stories42M.bin
==6544== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.56%  4.35212s     17898  243.16us     799ns  17.876ms  [CUDA memcpy HtoD]
                    8.03%  3

In [49]:
#@title Generate Stories

# Generate args
max_token = 256 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
prompt = "One day, Lily met a Shoggoth" #@param {type:"string"}

print(f"model: {model_file}, max_token: {max_token}, temperature: {temperature}, top_p: {top_p}, prompt: {prompt}")
print(f"----------------------------\n")

cmd = f'./run {model_file} -t {temperature} -p {top_p} -n {max_token} -i "{prompt}"'
!{cmd}

model: stories42M.bin, max_token: 256, temperature: 0.8, top_p: 0.9, prompt: One day, Lily met a Shoggoth
----------------------------

One day, Lily met a Shoggoth. He was very shiny and sweet. He said to Lily, "I have a special gift for you. It's a horn." Lily was so excited. She couldn't believe it. She asked the Shogggoth, "What does it do?" The Shoggoth replied, "You have to blow it and make a loud noise." Lily was delighted. She blew the horn and it made a loud noise. She smiled and said, "Wow, it's so loud!" The Shogggoth smiled and said, "You're very welcome. Have a nice day!" Lily thanked the Shogggest and she ran off, skipping happily.
achieved tok/s: 322.134387


In [ ]:



# 详细采集（包含 CUDA、cuBLAS、cuDNN、NVTX、OS 运行时）
!nsys profile -t cuda,cublas,cudnn,nvtx,osrt --stats=true -o yyj_report ./run stories42M.bin


Once upon a time, there was a little girl named Lily. One day, Lily's mommy asked her to help mop the floor. Lily was excited to help her mommy, so she grabbed the mop and started to clean. 
As she was cleaning, she noticed that the floor was very slippery. She was careful not to fall as she mops. Suddenly, she heard her mommy's phone ring. Her mommy answered the phone and talked for a while. 
Lily continued to mop the floor, but she started to feel dizzy. She stopped cleaning and sat down on the couch. Her mommy noticed that she was feeling dizzy and asked her if she wanted to take a break. Lily nodded her head and her mommy gave her some water to drink. After a few sips, Lily felt much better. She finished mopping the floor and her mommy thanked her for being such a good helper.
achieved tok/s: 21.048490
Generating '/tmp/nsys-report-5837.qdstrm'
[1/8] [========================100%] yyj_report.nsys-rep
[2/8] [========================100%] yyj_report.sqlite
[3/8] Executing 'nvtx_sum' s